<모듈 불러오기>

In [0]:
import cv2
import torch
import torch.nn as nn
from torchvision.models import vgg16
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image


<GPU 확인>

In [0]:
device = torch.device('cuda')

<VGG 모델 정의>

In [0]:
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()

        # get the pretrained VGG19 network
        self.vgg = vgg16(pretrained=True)

        # disect the network to access its last convolutional layer
        self.features_conv = self.vgg.features[:36]

        # get the classifier of the vgg19
        self.classifier = self.vgg.classifier

        # placeholder for the gradients
        self.gradients = None

    # hook for the gradients of the activations
    def activations_hook(self, grad):
        self.gradients = grad

    def forward(self, x):
        x = self.features_conv(x)

        # register the hook
        h = x.register_hook(self.activations_hook)
        x = x.view((1, -1))
        x = self.classifier(x)
        return x

    # method for the gradient extraction
    def get_activations_gradient(self):
        return self.gradients

    # method for the activation extraction
    def get_activations(self, x):
        return self.features_conv(x)

<모델 선언>

In [0]:
# initialize the VGG model
vgg = VGG().to(device)

# set the evaluation mode
vgg.eval()

<이미지 불러오기>

In [0]:
img = Image.open("cute.jpg")
img = np.asarray(img)
img = cv2.resize(img, (224, 224))
img = img.transpose([2, 0, 1])
img = img.astype('float32')
img /= 255.


<이미지 모델에 전달하기 (예측)>

In [0]:
img = np.expand_dims(img, 0)
img = torch.tensor(img).to(device)
pred = vgg(img)
pred.argmax(dim=1)

<Imagenet 데이터 클래스명 확인>

In [0]:
predicted = pred.cpu().detach().numpy()[0]
candidate = predicted.argsort()[-5:][::-1]

import json
class_idx = json.load(open("imagenet_class_index.json"))
class_idx['285']

for c in candidate:
    print(class_idx[str(c)])

<grad-CAM 코드>

In [0]:
# get the gradient of the output with respect to the parameters of the model
pred[:, 285].backward()

# pull the gradients out of the model
gradients = vgg.get_activations_gradient()

# pool the gradients across the channels
pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])

# get the activations of the last convolutional layer
activations = vgg.get_activations(img).detach()

# weight the channels by corresponding gradients
for i in range(512):
    activations[:, i, :, :] *= pooled_gradients[i]

# average the channels of the activations
heatmap = torch.mean(activations, dim=1).squeeze()

# relu on top of the heatmap
# expression (2) in https://arxiv.org/pdf/1610.02391.pdf
heatmap = np.maximum(heatmap.to('cpu'), 0)

# normalize the heatmap
heatmap /= torch.max(heatmap)

# draw the heatmap
plt.matshow(heatmap.squeeze())
plt.show()

<히트맵 그리기>

In [0]:
img = Image.open("cute.jpg")
img = np.asarray(img)
img = cv2.resize(img, (224, 224))

heatmap = cv2.resize(heatmap.numpy(), (img.shape[1], img.shape[0]))
heatmap = np.uint8(255 * heatmap)
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
superimposed_img = heatmap * 0.4 + img
cv2.imwrite('map.jpg', superimposed_img)